In [2]:
# default_exp data.ts_loader

In [3]:
#hide
%load_ext autoreload
%autoreload 2

In [4]:
#export
import numpy as np
import pandas as pd
import random
import torch as t
import copy
from fastcore.foundation import patch
from nixtla.data.ts_dataset import TimeSeriesDataset
from collections import defaultdict

In [5]:
#export
class TimeSeriesLoader(object):
    def __init__(self,
                 ts_dataset:TimeSeriesDataset,
                 model:str,
                 offset:int,
                 window_sampling_limit: int, 
                 input_size: int,
                 output_size: int,
                 idx_to_sample_freq: int, #TODO: not active yet
                 batch_size: int,
                 ts_outsample_mask: list=[]):
        """
        """
        self.model = model
        self.window_sampling_limit = window_sampling_limit
        self.input_size = input_size
        self.output_size = output_size
        self.batch_size = batch_size
        self.idx_to_sample_freq = idx_to_sample_freq
        self.offset = offset
        self.ts_dataset = copy.deepcopy(ts_dataset) #TODO: sacar deep_copy
        self.ts_outsample_mask = ts_outsample_mask
        self.t_cols = self.ts_dataset.t_cols

        # Overwrite mask if provided
        if len(self.ts_outsample_mask) > 0:
            self.ts_dataset.ts_tensor[:, self.t_cols.index('outsample_mask'), :] = t.as_tensor(ts_outsample_mask,dtype=t.float32)
       
        # Create rolling window matrix and broadcasted x_s
        self._create_train_data()
        self._is_train = True

        #TODO: cambiar estos prints
        # print('X: time series features, of shape (#series,#times,#features): \t' + str(X.shape))
        # print('Y: target series (in X), of shape (#series,#times): \t \t' + str(Y.shape))
        # print('S: static features, of shape (#series,#features): \t \t' + str(S.shape))

    def _update_sampling_windows_idxs(self):
        # Only sample during training windows with at least one active output mask
        sampling_idx = t.sum(self.ts_windows[:, self.t_cols.index('outsample_mask'), -self.output_size:], axis=1)
        sampling_idx = t.nonzero(sampling_idx > 0)
        return list(sampling_idx.flatten().numpy())

    def _create_windows_tensor(self):
        """
        Comment here
        TODO: Cuando creemos el otro dataloader, si es compatible lo hacemos funcion transform en utils
        """
        tensor, right_padding = self.ts_dataset.get_filtered_tensor(offset=self.offset, output_size=self.output_size,
                                                                    window_sampling_limit=self.window_sampling_limit)
        tensor = t.Tensor(tensor)
        _, n_channels, _ = tensor.size()

        padder = t.nn.ConstantPad1d(padding=(self.input_size-1, right_padding), value=0)
        tensor = padder(tensor)

        # Last output_size outsample_mask and y to 0
        tensor[:, self.t_cols.index('y'), -self.output_size:] = 0 # overkill to ensure no leakage
        tensor[:, self.t_cols.index('outsample_mask'), -self.output_size:] = 0

        # Creating rolling windows
        windows = tensor.unfold(dimension=-1, size=self.input_size + self.output_size, step=1)
        windows = windows.permute(2,0,1,3)
        windows = windows.reshape(-1, n_channels, self.input_size + self.output_size)
        return windows

    def __len__(self):
        return len(self.len_series)

    def __iter__(self):
        #TODO: revisar como se hace el -1 de batch_size en un dataloader de torch. Otra opcion es simplemente batch_size grande,
        # tambien se puede arregar con epoca
        while True:
            if self._is_train:
                if self.batch_size > 0:
                    sampled_ts_indices = np.random.choice(self.windows_sampling_idx, size=self.batch_size, replace=True)
                else:
                    sampled_ts_indices = self.windows_sampling_idx
            else:
                # Get last n_series windows, dataset is ordered because of unfold
                sampled_ts_indices = list(range(self.n_windows-self.ts_dataset.n_series, self.n_windows))

            batch = self.__get_item__(sampled_ts_indices)

            yield batch

    def __get_item__(self, index):
        if self.model == 'nbeats':
            return self._nbeats_batch(index)
        elif self.model == 'esrnn':
            assert 1<0, 'hacer esrnn'
        else:
            assert 1<0, 'error'

    def _nbeats_batch(self, index):
        windows = self.ts_windows[index]
        x_s = self.x_s[index]

        insample_y = windows[:, self.t_cols.index('y'), :self.input_size]
        insample_x_t = windows[:, (self.t_cols.index('y')+1):self.t_cols.index('insample_mask'), :self.input_size]
        insample_mask = windows[:, self.t_cols.index('insample_mask'), :self.input_size]

        outsample_y = windows[:, self.t_cols.index('y'), self.input_size:]
        outsample_x_t = windows[:, (self.t_cols.index('y')+1):self.t_cols.index('insample_mask'), self.input_size:]
        outsample_mask = windows[:, self.t_cols.index('outsample_mask'), self.input_size:]

        batch = {'insample_y':insample_y, 'insample_x_t':insample_x_t, 'insample_mask':insample_mask,
                  'outsample_y':outsample_y, 'outsample_x_t':outsample_x_t, 'outsample_mask':outsample_mask,
                  'x_s':x_s}

        return batch

    def _create_train_data(self):
        """
        """
        #print('Creating windows matrix ...')
        # Create rolling window matrix
        self.ts_windows = self._create_windows_tensor()
        self.n_windows = len(self.ts_windows)
        # Broadcast x_s: This works because unfold in windows_tensor, padded windows, unshuffled data.
        self.x_s = self.ts_dataset.x_s.repeat(int(self.n_windows/self.ts_dataset.n_series), 1)
        self.windows_sampling_idx = self._update_sampling_windows_idxs()

    def update_offset(self, offset):
        if offset == self.offset:
            return # Avoid extra computation
        self.offset = offset
        self._create_train_data()

    def get_meta_data_var(self, var):
        """
        """
        return self.ts_dataset.get_meta_data_var(var)

    def get_n_variables(self):
        return self.ts_dataset.n_x_t, self.ts_dataset.n_s_t

    def get_n_series(self):
        return self.ts_dataset.n_series

    def get_max_len(self):
        return self.ts_dataset.max_len

    def get_n_channels(self):
        return self.ts_dataset.n_channels

    def get_frequency(self):
        return self.ts_dataset.frequency

    def train(self):
        self._is_train = True

    def eval(self):
        self._is_train = False

In [6]:
from nixtla.data.datasets.epf import EPF,EPFInfo

In [7]:
ts_dataset = EPF.load('data', group='NP')

100%|██████████| 2.12M/2.12M [00:01<00:00, 1.80MiB/s]
INFO:nixtla.data.datasets.utils:Successfully downloaded NP.csv, 2118914, bytes.
100%|██████████| 2.53M/2.53M [00:01<00:00, 1.95MiB/s]
INFO:nixtla.data.datasets.utils:Successfully downloaded PJM.csv, 2530311, bytes.
100%|██████████| 1.97M/1.97M [00:01<00:00, 1.87MiB/s]
INFO:nixtla.data.datasets.utils:Successfully downloaded BE.csv, 1974555, bytes.
100%|██████████| 2.00M/2.00M [00:01<00:00, 1.72MiB/s]
INFO:nixtla.data.datasets.utils:Successfully downloaded FR.csv, 1996871, bytes.
100%|██████████| 2.10M/2.10M [00:01<00:00, 1.81MiB/s]
INFO:nixtla.data.datasets.utils:Successfully downloaded DE.csv, 2095831, bytes.
Processing dataframes ...
Creating ts tensor ...


In [9]:
ts_dataset.ts_tensor

tensor([[[3.1050e+01, 3.0470e+01, 2.8920e+01,  ..., 2.6820e+01,
          2.6650e+01, 2.5680e+01],
         [4.2497e+04, 4.1463e+04, 4.0812e+04,  ..., 4.5471e+04,
          4.4386e+04, 4.3017e+04],
         [2.7980e+03, 2.4170e+03, 2.0360e+03,  ..., 2.1290e+03,
          1.8270e+03, 1.6890e+03],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 1.0000e+00,
          1.0000e+00, 1.0000e+00],
         [1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 1.0000e+00,
          1.0000e+00, 1.0000e+00]]])